<a href="https://colab.research.google.com/github/lisa11323/BNPL/blob/main/%EC%86%8C%ED%96%89%EB%A1%A0_%EB%85%BC%EB%AC%B8_%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 전처리 코드 (반드시 리뷰 칼럼 명이 "Review"여야함)

1.1 zero-shot 분류용 전처리 코드
**"여기서 나온 결과 엑셀 파일을 결과를 아래 1.2 BERTopic용 전처리 코드에 업로드 해서 돌리기"**

In [ ]:
# 비영어+이모지 제거+음절 전처리 후 필터링
!pip install -U pandas numpy
!pip install nltk emoji

import pandas as pd
import re
import emoji
from tqdm import tqdm
from google.colab import files

# 📁 파일 업로드
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# ⏳ tqdm과 pandas 연동
tqdm.pandas()

# 📄 엑셀 파일 읽기
df = pd.read_excel(file_name)

# 🧹 이모지 + 비영어 문자 제거 함수
def clean_text(text):
    if isinstance(text, float):
        text = str(text)
    text = emoji.replace_emoji(text, replace="")
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text.strip()

# 🔄 전처리 적용 (기존 'Review' 덮어쓰기)
df['Review'] = df['Review'].progress_apply(clean_text)

# ✨ 리뷰 길이 계산 함수
def get_review_length(text):
    try:
        return len(str(text).split())
    except:
        return 0

# 📏 Review_length 컬럼 생성
df['Review_length'] = df['Review'].progress_apply(get_review_length)

# 🔘 옵션: 리뷰 길이 필터링 여부
filter_by_length = True    # True → 길이 기준으로 필터링 / False → 필터링 하지 않음
min_length = 10            # 기준 길이

if filter_by_length:
    df = df[df['Review_length'] >= min_length].reset_index(drop=True)

# ✅ 엑셀로 저장
preprocessed_file_path = 'preprocessed_comments_with_length.xlsx'
df.to_excel(preprocessed_file_path, index=False)
print(f"✅ 전처리 + 리뷰 길이 필터링({filter_by_length}) 데이터를 '{preprocessed_file_path}' 파일로 저장했습니다.")

# 📥 다운로드
files.download(preprocessed_file_path)


  Using cached emoji-2.14.1-py3-none-any.whl.metadata (5.7 kB)
Using cached emoji-2.14.1-py3-none-any.whl (590 kB)


Saving (2025.05.02기준 전체 리뷰Raw_Data) playstore_chatgpt_reviews.xlsx to (2025.05.02기준 전체 리뷰Raw_Data) playstore_chatgpt_reviews.xlsx


100%|██████████| 761264/761264 [00:01<00:00, 744602.72it/s] 


✅ 전처리 + 리뷰 길이 필터링(True) 데이터를 'preprocessed_comments_with_length.xlsx' 파일로 저장했습니다.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

1.2 BERTopic 용 전처리 코드

In [ ]:
!pip install tqdm
!pip install nltk
!pip install sentence-transformers
!pip install hdbscan
!pip install plotly

# 1. 라이브러리 임포트
import pandas as pd
from tqdm import tqdm
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from google.colab import files
import io

# 2. Plotly 설정 (필요 시)
import plotly.io as pio
pio.renderers.default = "colab"

# 3. NLTK 리소스 다운로드
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# 4. 파일 업로드
print("📁 엑셀 파일을 업로드 해주세요")
uploaded = files.upload()

# 5. 파일 읽기
for fn in uploaded.keys():
    file_path = fn

df = pd.read_excel(file_path)
print(f"✅ 파일 로드 완료: {file_path}")

# 6. 불용어 및 표제어 처리 준비
default_stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# 7. 전처리 함수 정의 (영어만 남기도록 수정됨)
def preprocess_text(text):
    if pd.isna(text): return ""
    text = text.lower()
    text = re.sub(r"[^a-z\s]", " ", text)  # 영어 소문자와 공백만 남김
    text = re.sub(r"\s+", " ", text).strip()
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in default_stopwords]
    return " ".join(tokens)

# 🔄 8. Review 열 전처리
if "Review" in df.columns:
    tqdm.pandas()
    # 기존 Review를 Review_Raw로 이름 변경
    df = df.rename(columns={"Review": "Review_Raw"})
    # 전처리한 결과를 comment_preprocessed에 저장
    df["Review_preprocessed"] = df["Review_Raw"].progress_apply(preprocess_text)
    print("✅ 전처리 완료")
else:
    print("❌ 'Review' 열이 존재하지 않습니다.")

# 9. 엑셀로 저장 및 다운로드
output_file = "preprocessed_Review.xlsx"
df.to_excel(output_file, index=False)
files.download(output_file)

📁 엑셀 파일을 업로드 해주세요


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Saving preprocessed_comments_with_length.xlsx to preprocessed_comments_with_length (1).xlsx
✅ 파일 로드 완료: preprocessed_comments_with_length (1).xlsx


100%|██████████| 124661/124661 [00:10<00:00, 11472.53it/s]


✅ 전처리 완료


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 2. BERTopic + Louvain 알고리즘 코드

2.1 필요한 라이브러리 다운로드

In [ ]:
!pip install pandas openpyxl bertopic tqdm hdbscan plotly umap-learn matplotlib python-louvain

2.2 BERTopic 모델 로드

In [ ]:
# 1. 라이브러리 임포트
import pandas as pd
from bertopic import BERTopic
from tqdm import tqdm
import re
import string
from google.colab import files
import hdbscan
from umap import UMAP
import plotly.io as pio
pio.renderers.default = "colab"  # plotly 시각화 설정

# 2. 파일 업로드
print("📁 엑셀 파일을 업로드 해주세요")
uploaded = files.upload()

# 3. 파일 읽기
for fn in uploaded.keys():
    file_path = fn

df = pd.read_excel(file_path)
print(f"✅ 파일 로드 완료: {file_path}")
print("📊 Review_preprocessed 열 미리보기:")
print(df["Review_preprocessed"].head())

# 4. 텍스트 전처리 함수
def clean_text(text):
    text = text.lower()
    text = re.sub(rf"[{re.escape(string.punctuation)}]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# 5. 텍스트 전처리
comments = df["Review_preprocessed"].dropna().astype(str).tolist()
print("🧹 텍스트 전처리 중...")
comments_cleaned = [clean_text(text) for text in tqdm(comments)]

# 6. hDBSCAN 설정
hdbscan_model = hdbscan.HDBSCAN(
    min_cluster_size=100,
    min_samples=5,
    metric='euclidean',
    prediction_data=True
)

# ✅ 7. 커스터마이즈된 UMAP 모델 설정
umap_model = UMAP(
    n_neighbors=15,
    n_components=5,
    min_dist=0.1,
    metric='cosine',
    random_state=42
)

# ✅ 8. BERTopic 모델 학습 (embedding_model 미설정, 기본값 사용)
print("🤖 BERTopic + hDBSCAN + custom UMAP 훈련 중...")
topic_model = BERTopic(
    hdbscan_model=hdbscan_model,
    umap_model=umap_model,
    language="english",
    verbose=True
)
topics, probs = topic_model.fit_transform(comments_cleaned)

# 9. 시각화
fig = topic_model.visualize_topics()
fig.show()

# 10. 원본 데이터에 토픽 ID 붙이기
df_result = df.copy()
df_result["topic_id"] = topics

# 11. 저장 파일 이름 정의 및 저장
output_file = "bertopic_result.xlsx"
df_result.to_excel(output_file, index=False)

# 12. 다운로드 제공
print("💾 분석 결과를 엑셀로 저장했습니다. 다운로드 버튼을 누르세요.")
files.download(output_file)


📁 엑셀 파일을 업로드 해주세요


Saving preprocessed_Review.xlsx to preprocessed_Review (1).xlsx
✅ 파일 로드 완료: preprocessed_Review (1).xlsx
📊 Review_preprocessed 열 미리보기:
0    epic love ai great company fun hang write crea...
1    thankful creator cant put word much helped u t...
2                    good perfect friend nice job done
3        amazing create image picture idea chat useful
4                      app good chatgpt happy asistent
Name: Review_preprocessed, dtype: object
🧹 텍스트 전처리 중...


100%|██████████| 124661/124661 [00:01<00:00, 106830.42it/s]
2025-08-01 14:05:05,826 - BERTopic - Embedding - Transforming documents to embeddings.


🤖 BERTopic + hDBSCAN 훈련 중...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/3896 [00:00<?, ?it/s]

2025-08-01 14:06:00,678 - BERTopic - Embedding - Completed ✓
2025-08-01 14:06:00,679 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-08-01 14:09:51,469 - BERTopic - Dimensionality - Completed ✓
2025-08-01 14:09:51,473 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-08-01 14:10:05,251 - BERTopic - Cluster - Completed ✓
2025-08-01 14:10:05,291 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-08-01 14:10:06,844 - BERTopic - Representation - Completed ✓


💾 분석 결과를 엑셀로 저장했습니다. 다운로드 버튼을 누르세요.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

토픽 정보 추출

In [ ]:
import pandas as pd
from google.colab import files

# 📋 토픽 정보 가져오기
topic_info = topic_model.get_topic_info()

# 📋 pandas 출력 옵션 (선택)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.colheader_justify", "center")

# 📋 코랩에서 표 형태로 출력
from IPython.display import display
display(topic_info)

# 🔷 엑셀로 저장
output_path = "topic_info.xlsx"
topic_info.to_excel(output_path, index=False)
print(f"✅ 토픽 정보가 엑셀로 저장되었습니다: {output_path}")

# 🔷 코랩에서 다운로드
files.download(output_path)


,Topic,Count,Name,Representation,Representative_Docs
0,-1,46134,-1_chatgpt_answer_time_like,"[chatgpt, answer, time, like, question, give, ...",[perfect app question need give correct answer...
1,0,4236,0_image_picture_photo_upload,"[image, picture, photo, upload, generate, pic,...","[good take much time generate photo, nice app ..."
2,1,4007,1_voice_audio_speech_feature,"[voice, audio, speech, feature, mode, text, so...","[think best voice chat, awesome please add tex..."
3,2,2914,2_ai_tool_best_ever,"[ai, tool, best, ever, world, artificial, inte...","[ai best ai ever seen, good ai love, ai good l..."
4,3,2133,3_ai_best_ever_app,"[ai, best, ever, app, apps, seen, used, open, ...","[best ai app ever used, best ai app ever use, ..."
5,4,1912,4_gpt_chat_thank_love,"[gpt, chat, thank, love, friend, thanks, help,...","[love chat gpt use time, really like chat gpt ..."
6,5,1790,5_chatgpt_friend_love_thank,"[chatgpt, friend, love, thank, life, thanks, b...","[best best best good love chatgpt best friend,..."
7,6,1720,6_hai_bahut_bhi_se,"[hai, bahut, bhi, se, ye, ki, ke, aur, yah, ka]",[assalamu alaykum mera naam muhammad awais hai...
8,7,1636,7_chat_history_message_delete,"[chat, history, message, delete, previous, con...",[great app use alot like mean alot im problem ...
9,8,1515,8_answer_question_ask_app,"[answer, question, ask, app, give, answering, ...","[good app answer question, app answer question..."


✅ 토픽 정보가 엑셀로 저장되었습니다: topic_info.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

2.3 각 키워드별 대표 키워드 추출

In [ ]:
# 🧠 1. topic_id별 대표 키워드 출력
print("🧠 각 topic_id별 대표 키워드:")
for topic_id in topic_model.get_topics().keys():
    if topic_id == -1:
        continue
    print(f"\n🟦 Topic {topic_id}:")
    for word, weight in topic_model.get_topic(topic_id)[:10]:
        print(f"  - {word} ({weight:.4f})")

# 🧠 2. 토픽 임베딩을 이용한 유사도 행렬 수동 생성
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 토픽 임베딩 추출 (각 토픽을 임베딩한 벡터)
embeddings = topic_model.topic_embeddings_

# 유사도 행렬 생성 (cosine similarity)
similarity_matrix = cosine_similarity(embeddings)


2.4 문장 클러스터 시각화

In [ ]:
# 1. 직접 SentenceTransformer 선언 (BERTopic 학습 시 사용한 것과 동일한 모델)
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # BERTopic 기본 모델

# 2. 문장 임베딩 벡터 생성
embeddings = embedding_model.encode(comments_cleaned, show_progress_bar=True)

# 3. UMAP 차원 축소
from umap import UMAP
import matplotlib.pyplot as plt
import numpy as np

umap_model = UMAP(n_components=2, random_state=42)
embeddings_2d = umap_model.fit_transform(embeddings)

# 4. 시각화
unique_topics = sorted(set(topics))
colors = plt.cm.get_cmap('tab20', len(unique_topics))

plt.figure(figsize=(10, 8))
for topic in unique_topics:
    idx = [i for i, t in enumerate(topics) if t == topic]
    plt.scatter(embeddings_2d[idx, 0], embeddings_2d[idx, 1],
                label=f"Topic {topic}", alpha=0.5, s=30, color=colors(topic))

plt.title("🔍 문장 클러스터 시각화 (UMAP + Topic Color)")
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.grid(True)
plt.show()

2.5 hierarchical_topics 시각화

In [ ]:
print("🌳 Hierarchical topics 계산 중...")

# linkage 함수 정의
from scipy.cluster import hierarchy as sch

linkage_function = lambda x: sch.linkage(x, method='single', optimal_ordering=True)

hierarchical_topics = topic_model.hierarchical_topics(
    comments_cleaned,
    linkage_function=linkage_function
)

print("🌳 계층형 토픽 시각화 및 PNG 저장 중...")

# Plotly로 시각화 (추천)
fig_hierarchy = topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)
fig_hierarchy.show()


2.6 BERTopic 시각화

In [ ]:
# 토픽별 시각화
print("📊 토픽별 시각화 중...")
fig_topics = topic_model.visualize_topics()

# 화면에 표시
fig_topics.show()

2.7 Barchart 시각화

In [ ]:
# 막대그래프 시각화
print("📊 토픽별 키워드 막대그래프 시각화 중...")
fig_barchart = topic_model.visualize_barchart(top_n_topics=137)

# 화면에 표시
fig_barchart.show()

2.8 heatmap 시각화

In [ ]:
# heatmap 시각화
print("📊 토픽 간 유사도 heatmap 시각화 중...")
fig_heatmap = topic_model.visualize_heatmap()
# 화면에 표시
fig_heatmap.show()

2.9 Louvain 알고리즘 모델 불러오기

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import community.community_louvain as community_louvain
from tqdm import tqdm
import numpy as np

# 0. 모델 준비
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# 1. 토픽별 평균 임베딩 생성
print("📌 [1/5] 토픽별 평균 임베딩 생성 중...")
topic_ids = sorted(set(topics))
topic_vectors = []

for tid in tqdm(topic_ids, desc="▶️ 토픽 임베딩 중"):
    idx = [i for i, t in enumerate(topics) if t == tid]
    topic_sentences = [comments_cleaned[i] for i in idx]
    if topic_sentences:
        topic_embedding = embedding_model.encode(topic_sentences)
        topic_mean = np.mean(topic_embedding, axis=0)
        topic_vectors.append(topic_mean)
    else:
        topic_vectors.append(np.zeros(embedding_model.get_sentence_embedding_dimension()))

# 2. 유사도 행렬 계산
print("🔗 [2/5] 토픽 간 유사도 계산 중...")
similarity_matrix = cosine_similarity(topic_vectors)

# 3. 토픽 간 그래프 생성
print("🔗 [3/5] 유사도 기반 그래프 생성 중...")
G = nx.Graph()
for i in tqdm(range(len(topic_ids)), desc="▶️ 그래프 엣지 생성 중"):
    for j in range(i + 1, len(topic_ids)):
        weight = similarity_matrix[i][j]

        if weight > 0.5:  # 임계값은 필요 시 조절 가능
          G.add_edge(topic_ids[i], topic_ids[j], weight=weight)

# 4. Louvain community detection
print("🧠 [4/5] Louvain 커뮤니티 탐지 중...")
partition = community_louvain.best_partition(G)

# 💡 고립된 토픽도 메타토픽으로 넣기
isolated_topics = list(set(topic_ids) - set(partition.keys()))
if isolated_topics:
    next_meta_topic = max(partition.values()) + 1 if partition else 0
    for iso_tid in isolated_topics:
        partition[iso_tid] = next_meta_topic
        next_meta_topic += 1

# 5. 메타토픽 정리 및 출력
print("📁 [5/5] 메타토픽 정리 중...")
meta_topic_dict = {}
for topic_id, community_id in partition.items():
    if community_id not in meta_topic_dict:
        meta_topic_dict[community_id] = []
    meta_topic_dict[community_id].append(topic_id)

# 결과 출력
print("\n🧠 Louvain 기반 메타토픽 그룹:")
for group, topics_in_group in meta_topic_dict.items():
    print(f"\n🔷 메타토픽 {group}:")
    print(f"Topic IDs: {topics_in_group}")
    for tid in topics_in_group:
        words = topic_model.get_topic(tid)[:5]
        keywords = ", ".join([w[0] for w in words])
        print(f"  - Topic {tid}: {keywords}")

In [ ]:
import pandas as pd

# 원본 데이터에 topic_id 추가
df_with_topic = df.copy()
df_with_topic["topic_id"] = topics

# topic_id → meta_topic_id 매핑
topic_to_meta = partition  # Louvain 결과

# topic_id 기준으로 meta_topic_id 컬럼 생성
df_with_topic["meta_topic_id"] = df_with_topic["topic_id"].map(topic_to_meta)

# -1 (노이즈) 제거
df_with_topic = df_with_topic[df_with_topic["topic_id"] != -1]

# 원하는 순서로 정렬
df_with_topic = df_with_topic.sort_values(by=["meta_topic_id", "topic_id"]).reset_index(drop=True)

# 결과 미리보기
print("📄 메타토픽별 리뷰 데이터:")
print(df_with_topic[["meta_topic_id", "topic_id", "Review_preprocessed"]].head())

# 엑셀로 저장
output_file = "meta_topic_grouped_reviews.xlsx"
df_with_topic.to_excel(output_file, index=False)

# 다운로드
from google.colab import files
files.download(output_file)


In [ ]:
# 📌 토픽-메타토픽-키워드(+빈도수) 테이블 생성
rows = []

for topic_id, meta_topic in partition.items():
    if topic_id == -1:
        continue  # 노이즈 제외

    # 각 topic_id에서 상위 10개 키워드 및 가중치(빈도수 유사값) 추출
    keywords = topic_model.get_topic(topic_id)

    for word, weight in keywords[:10]:
        rows.append({
            "meta_topic_id": meta_topic,
            "topic_id": topic_id,
            "keyword": word,
            "score": round(weight, 4)  # score 또는 weight로 표시
        })

# 데이터프레임으로 변환
df_keywords = pd.DataFrame(rows)

# 정렬
df_keywords = df_keywords.sort_values(by=["meta_topic_id", "topic_id", "score"], ascending=[True, True, False]).reset_index(drop=True)

# 엑셀로 저장 (선택적)
output_keywords_file = "meta_topic_keywords_with_scores.xlsx"
df_keywords.to_excel(output_keywords_file, index=False)

# 다운로드 (Colab 환경에서만 사용)
from google.colab import files
files.download(output_keywords_file)


2.10 메타토픽 대표 키워드 30개 새로 추출

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from collections import defaultdict
import numpy as np
import pandas as pd

# 1. 각 메타토픽별 문서 그룹 생성
print("📌 [1/4] 메타토픽별 문장 그룹 생성 중...")
meta_topic_docs = defaultdict(list)

for comment, topic_id in zip(comments_cleaned, topics):
    if topic_id == -1:
        continue
    meta_topic = partition.get(topic_id, -1)
    if meta_topic != -1:
        meta_topic_docs[meta_topic].append(comment)

# 2. 메타토픽별 문서 결합
print("📌 [2/4] 메타토픽별 문서 결합 중...")
meta_topic_texts = []
meta_topic_ids = []

for meta_topic_id, doc_list in meta_topic_docs.items():
    merged_text = " ".join(doc_list)
    meta_topic_texts.append(merged_text)
    meta_topic_ids.append(meta_topic_id)

# 3. C-TF-IDF 계산
print("📊 [3/4] C-TF-IDF 계산 중...")
vectorizer = CountVectorizer(stop_words="english")
X = vectorizer.fit_transform(meta_topic_texts)

ctfidf = ClassTfidfTransformer()
X_ctfidf = ctfidf.fit_transform(X)  # ✅ multiplier 제거

# 4. 상위 키워드 추출
print("🏷️ [4/4] 메타토픽별 키워드 추출 중...")
terms = vectorizer.get_feature_names_out()
meta_topic_keywords = []

for idx, row in enumerate(X_ctfidf):
    meta_topic_id = meta_topic_ids[idx]
    row_data = row.toarray().flatten()
    top_indices = row_data.argsort()[::-1][:30]

    for rank, i in enumerate(top_indices, 1):
        meta_topic_keywords.append({
            "meta_topic_id": meta_topic_id,
            "rank": rank,
            "keyword": terms[i],
            "ctfidf_score": round(row_data[i], 6)
        })

# 결과 저장
df_meta_keywords = pd.DataFrame(meta_topic_keywords)
df_meta_keywords = df_meta_keywords.sort_values(by=["meta_topic_id", "rank"])

# 🔽 엑셀 저장
final_output = "meta_topic_top30_keywords_ctfidf.xlsx"
df_meta_keywords.to_excel(final_output, index=False)

# 🔽 다운로드 (Colab)
from google.colab import files
files.download(final_output)


2.11 BERTchart 처럼 최종 메타토픽 키워드 30개 시각화

In [ ]:
import pandas as pd
import plotly.express as px

# 1. 엑셀 파일 로드
df_ctfidf = pd.read_excel("meta_topic_top30_keywords_ctfidf.xlsx")

# 2. 특정 메타토픽만 시각화하고 싶다면 필터링 (예: meta_topic_id = 0)
# meta_id = 0
# df_plot = df_ctfidf[df_ctfidf["meta_topic_id"] == meta_id]

# 모든 메타토픽을 하나씩 시각화 (반복 루프)
meta_ids = sorted(df_ctfidf["meta_topic_id"].unique())

for meta_id in meta_ids:
    df_plot = df_ctfidf[df_ctfidf["meta_topic_id"] == meta_id].sort_values(by="ctfidf_score", ascending=True)

    fig = px.bar(
        df_plot,
        x="ctfidf_score",
        y="keyword",
        orientation='h',
        title=f"Meta-topic {meta_id} - Top 30 Keywords (C-TF-IDF)",
        labels={"ctfidf_score": "C-TF-IDF Score", "keyword": "Keyword"},
        height=700
    )

    fig.update_layout(
        yaxis=dict(tickfont=dict(size=12)),
        xaxis=dict(tickfont=dict(size=12)),
        margin=dict(l=120, r=20, t=50, b=40)
    )
    fig.show()


# 3. 독립변수 측정 코드 (메타토픽으로 나온 독립변수 4개 Zero-shot 분류로 비율 점수 계산 코드)

In [ ]:
# 📦 라이브러리 설치
!pip install scipy transformers pandas openpyxl tqdm matplotlib datasets scikit-learn --quiet

3.1 learning support

In [ ]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
from google.colab import files

# ⬆️ 엑셀 파일 업로드
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df = pd.read_excel(filename)

# ✅ 'Review_Raw' 열 확인
if 'Review_Raw' not in df.columns:
    raise ValueError("❌ 'Review_Raw' 열이 엑셀에 없습니다!")

# 🤖 Zero-shot 분류기 로드
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# 🎯 Learning Support 관련성 라벨 정의
learning_support_labels = [
    ("Completely Unrelated", 0.0),
    ("Slightly Unrelated", 0.25),
    ("Neutral", 0.5),
    ("Somewhat Related", 0.75),
    ("Strongly Related", 1.0)
]

# 🧠 Learning Support 관련성 점수 계산 함수
def classify_learning_support_score(comment):
    if pd.isna(comment) or not str(comment).strip():
        return 0.0

    labels = [label for label, _ in learning_support_labels]

    output = classifier(
        comment,
        candidate_labels=labels,
        hypothesis_template="This comment is {} to the learning support.",
        return_all_scores=True
    )

    if isinstance(output, list):
        result = output[0]  # return_all_scores=True 이므로 list of list
    elif isinstance(output, dict) and "scores" in output:
        result = [
            {'label': label, 'score': score}
            for label, score in zip(output["labels"], output["scores"])
        ]
    else:
        raise ValueError("❌ 모델 출력 형식이 예상과 다릅니다")

    weighted = sum(r['score'] * w for r, (_, w) in zip(result, learning_support_labels))
    return round(weighted, 4)

# 🏃 분석 실행
tqdm.pandas()
df["LearningSupportRelevance_score"] = df["Review_Raw"].progress_apply(classify_learning_support_score)

# 💾 저장 및 다운로드
output_file = filename.replace(".xlsx", "_learning_support_relevance_scored.xlsx")
df.to_excel(output_file, index=False)
files.download(output_file)

print("✅ 완료: LearningSupportRelevance_score (0~1) 계산 완료")


3.2 AI Task Assistant

In [ ]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
from google.colab import files

# ⬆️ 엑셀 파일 업로드
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df = pd.read_excel(filename)

# ✅ 'Review_Raw' 열 확인
if 'Review_Raw' not in df.columns:
    raise ValueError("❌ 'Review_Raw' 열이 엑셀에 없습니다!")

# 🤖 Zero-shot 분류기 로드
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# 🎯 AI Task Assistant 관련성 라벨 정의
ai_task_labels = [
    ("Completely Unrelated", 0.0),
    ("Slightly Unrelated", 0.25),
    ("Neutral", 0.5),
    ("Somewhat Related", 0.75),
    ("Strongly Related", 1.0)
]

# 🧠 AI Task Assistant 관련성 점수 계산 함수
def classify_ai_task_score(comment):
    if pd.isna(comment) or not str(comment).strip():
        return 0.0

    labels = [label for label, _ in ai_task_labels]

    output = classifier(
        comment,
        candidate_labels=labels,
        hypothesis_template="This comment is {} to the AI task assistant functionality.",
        return_all_scores=True
    )

    # output 검증 및 가중 평균 계산
    if isinstance(output, list):
        result = output[0]
    elif isinstance(output, dict) and "scores" in output:
        result = [
            {'label': label, 'score': score}
            for label, score in zip(output["labels"], output["scores"])
        ]
    else:
        raise ValueError("❌ 모델 출력 형식이 예상과 다릅니다")

    weighted = sum(r['score'] * w for r, (_, w) in zip(result, ai_task_labels))
    return round(weighted, 4)

# 🏃 분석 실행
tqdm.pandas()
df["AITaskAssistant_score"] = df["Review_Raw"].progress_apply(classify_ai_task_score)

# 💾 저장 및 다운로드
output_file = filename.replace(".xlsx", "_ai_task_assistant_scored.xlsx")
df.to_excel(output_file, index=False)
files.download(output_file)

print("✅ 완료: AITaskAssistant_score (0~1) 계산 완료")


3.3 chatbot interaction

In [ ]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
from google.colab import files

# ⬆️ 엑셀 파일 업로드
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df = pd.read_excel(filename)

# ✅ 'Review_Raw' 열 확인
if 'Review_Raw' not in df.columns:
    raise ValueError("❌ 'Review_Raw' 열이 엑셀에 없습니다!")

# 🤖 Zero-shot 분류기 로드
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# 🎯 Chatbot Interaction 관련성 라벨 정의
chatbot_labels = [
    ("Completely Unrelated", 0.0),
    ("Slightly Unrelated", 0.25),
    ("Neutral", 0.5),
    ("Somewhat Related", 0.75),
    ("Strongly Related", 1.0)
]

# 🧠 Chatbot Interaction 관련성 점수 계산 함수
def classify_chatbot_interaction_score(comment):
    if pd.isna(comment) or not str(comment).strip():
        return 0.0

    labels = [label for label, _ in chatbot_labels]

    output = classifier(
        comment,
        candidate_labels=labels,
        hypothesis_template="This comment is {} to the chatbot interaction.",
        return_all_scores=True
    )

    # output 검증 및 가중 평균 계산
    if isinstance(output, list):
        result = output[0]  # return_all_scores=True 이므로 list of list
    elif isinstance(output, dict) and "scores" in output:
        result = [
            {'label': label, 'score': score}
            for label, score in zip(output["labels"], output["scores"])
        ]
    else:
        raise ValueError("❌ 모델 출력 형식이 예상과 다릅니다")

    weighted = sum(r['score'] * w for r, (_, w) in zip(result, chatbot_labels))
    return round(weighted, 4)

# 🏃 분석 실행
tqdm.pandas()
df["ChatbotInteractionRelevance_score"] = df["Review_Raw"].progress_apply(classify_chatbot_interaction_score)

# 💾 저장 및 다운로드
output_file = filename.replace(".xlsx", "_chatbot_interaction_relevance_scored.xlsx")
df.to_excel(output_file, index=False)
files.download(output_file)

print("✅ 완료: ChatbotInteractionRelevance_score (0~1) 계산 완료")


Saving (50000 - 나머지) meta_topic_grouped_reviews.xlsx to (50000 - 나머지) meta_topic_grouped_reviews.xlsx


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
100%|██████████| 21776/21776 [44:53<00:00,  8.08it/s]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ 완료: ChatbotInteractionRelevance_score (0~1) 계산 완료


3.4 app malfunction

In [ ]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
from google.colab import files

# ⬆️ 엑셀 파일 업로드
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df = pd.read_excel(filename)

# ✅ 'Review_Raw' 열 확인
if 'Review_Raw' not in df.columns:
    raise ValueError("❌ 'Review_Raw' 열이 엑셀에 없습니다!")

# 🤖 Zero-shot 분류기 로드
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# 🎯 App Malfunction 관련성 라벨 정의
malfunction_labels = [
    ("Completely Unrelated", 0.0),
    ("Slightly Unrelated", 0.25),
    ("Neutral", 0.5),
    ("Somewhat Related", 0.75),
    ("Strongly Related", 1.0)
]

# 🧠 App Malfunction 관련성 점수 계산 함수
def classify_app_malfunction_score(comment):
    if pd.isna(comment) or not str(comment).strip():
        return 0.0

    labels = [label for label, _ in malfunction_labels]

    output = classifier(
        comment,
        candidate_labels=labels,
        hypothesis_template="This comment is {} to app malfunction issues.",
        return_all_scores=True
    )

    # output 검증 및 가중 평균 계산
    if isinstance(output, list):
        result = output[0]
    elif isinstance(output, dict) and "scores" in output:
        result = [
            {'label': label, 'score': score}
            for label, score in zip(output["labels"], output["scores"])
        ]
    else:
        raise ValueError("❌ 모델 출력 형식이 예상과 다릅니다")

    weighted = sum(r['score'] * w for r, (_, w) in zip(result, malfunction_labels))
    return round(weighted, 4)

# 🏃 분석 실행
tqdm.pandas()
df["AppMalfunction_score"] = df["Review_Raw"].progress_apply(classify_app_malfunction_score)

# 💾 저장 및 다운로드
output_file = filename.replace(".xlsx", "_app_malfunction_scored.xlsx")
df.to_excel(output_file, index=False)
files.download(output_file)

print("✅ 완료: AppMalfunction_score (0~1) 계산 완료")


# 4. 조절변수 측정 코드

4.1 humanlikeness zero-shot 측정 코드

In [ ]:
# 📚 라이브러리 불러오기
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
from google.colab import files

# ⬆️ 엑셀 파일 업로드
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# 📄 엑셀 데이터 불러오기
df = pd.read_excel(filename)

# ✅ 'comment' 열 존재 확인
if 'Review_Raw' not in df.columns:
    raise ValueError("❌ 'comment' 열이 엑셀에 없습니다!")

# 🤖 Zero-shot 분류기 로드
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# 🎯 Humanlikeness 라벨 정의
humanlikeness_labels = [
    ("Very Machianlike", 0.0),
    ("Somewhat Machianlike", 0.25),
    ("Neutral", 0.5),
    ("Somewhat Humanlike", 0.75),
    ("Very Humanlike", 1.0)
]

# 🧠 Humanlikeness 점수 계산 함수
def classify_humanlikeness_score(comment):
    if pd.isna(comment) or not str(comment).strip():
        return 0.0

    labels = [label for label, _ in humanlikeness_labels]

    output = classifier(
        comment,
        candidate_labels=labels,
        hypothesis_template="This comment mention to {} in terms of uncanny valley theory in conversational AI.",
        return_all_scores=True
    )

    # output 검증 및 가중 평균 계산
    if isinstance(output, list):
        result = output
    elif isinstance(output, dict) and "scores" in output:
        result = [
            {'label': label, 'score': score}
            for label, score in zip(output["labels"], output["scores"])
        ]
    else:
        raise ValueError("모델 출력 형식이 예상과 다릅니다")

    weighted = sum(r['score'] * w for r, (_, w) in zip(result, humanlikeness_labels))
    return round(weighted, 4)

# 🏃 분석 실행
tqdm.pandas()
df["Humanlikeness_score"] = df["Review_Raw"].progress_apply(classify_humanlikeness_score)

# 💾 결과 저장 및 다운로드
output_file = filename.replace(".xlsx", "_humanlikeness_scored.xlsx")
df.to_excel(output_file, index=False)
files.download(output_file)

print("✅ 완료: Humanlikeness_score (0~1)가 계산되어 저장되었습니다.")

4.2 familiarity zero-shot 측정 코드

In [ ]:
# 📚 라이브러리 불러오기
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
from google.colab import files

# ⬆️ 엑셀 파일 업로드
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# 📄 엑셀 데이터 불러오기
df = pd.read_excel(filename)

# ✅ 'comment' 열 존재 확인
if 'Review_Raw' not in df.columns:
    raise ValueError("❌ 'comment' 열이 엑셀에 없습니다!")

# 🤖 Zero-shot 분류기 로드
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# 🎯 Familiarity 라벨 정의 (5단계)
familiarity_labels = [
    ("Very Unfamiliar", 0.0),
    ("Somewhat Unfamiliar", 0.25),
    ("Neutral", 0.5),
    ("Somewhat Familiar", 0.75),
    ("Very Familiar", 1.0)
]

# 🧠 Familiarity 점수 계산 함수
def classify_familiarity_score(comment):
    if pd.isna(comment) or not str(comment).strip():
        return 0.0

    labels = [label for label, _ in familiarity_labels]

    output = classifier(
        comment,
        candidate_labels=labels,
        hypothesis_template="This comment mention to {} in terms of uncanny valley theory in conversational AI.",
        return_all_scores=True
    )

    # output 검증 및 가중 평균 계산
    if isinstance(output, list):
        result = output
    elif isinstance(output, dict) and "scores" in output:
        result = [
            {'label': label, 'score': score}
            for label, score in zip(output["labels"], output["scores"])
        ]
    else:
        raise ValueError("모델 출력 형식이 예상과 다릅니다")

    weighted = sum(r['score'] * w for r, (_, w) in zip(result, familiarity_labels))
    return round(weighted, 4)

# 🏃 분석 실행
tqdm.pandas()
df["Familiarity_score"] = df["Review_Raw"].progress_apply(classify_familiarity_score)

# 💾 결과 저장 및 다운로드
output_file = filename.replace(".xlsx", "_familiarity_scored.xlsx")
df.to_excel(output_file, index=False)
files.download(output_file)

print("✅ 완료: Familiarity_score (0~1)가 계산되어 저장되었습니다.")


# 5. 매개변수 측정 코드

5.1 주관성(subjectivity) zero-shot 측정 코드

In [ ]:
# 📚 라이브러리 불러오기
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
from google.colab import files

# ⬆️ 엑셀 파일 업로드
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# 📄 엑셀 데이터 불러오기
df = pd.read_excel(filename)

# ✅ 'comment' 열 존재 확인
if 'Review_Raw' not in df.columns:
    raise ValueError("❌ 'comment' 열이 엑셀에 없습니다!")

# 🤖 Zero-shot 분류기 로드
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# 🎯 Subjectivity 라벨 정의 (5단계)
subjectivity_labels = [
    ("Very Objective", 0.0),
    ("Somewhat Objective", 0.25),
    ("Neutral", 0.5),
    ("Somewhat Subjective", 0.75),
    ("Very Subjective", 1.0)
]

# 🧠 Subjectivity 점수 계산 함수
def classify_subjectivity_score(comment):
    if pd.isna(comment) or not str(comment).strip():
        return 0.0

    labels = [label for label, _ in subjectivity_labels]

    output = classifier(
        comment,
        candidate_labels=labels,
        hypothesis_template="This sentence expresses a {} opinion.",
        return_all_scores=True
    )

    # output 검증 및 가중 평균 계산
    if isinstance(output, list):
        result = output
    elif isinstance(output, dict) and "scores" in output:
        result = [
            {'label': label, 'score': score}
            for label, score in zip(output["labels"], output["scores"])
        ]
    else:
        raise ValueError("모델 출력 형식이 예상과 다릅니다")

    weighted = sum(r['score'] * w for r, (_, w) in zip(result, subjectivity_labels))
    return round(weighted, 4)

# 🏃 분석 실행
tqdm.pandas()
df["Subjectivity_score"] = df["Review_Raw"].progress_apply(classify_subjectivity_score)

# 💾 결과 저장 및 다운로드
output_file = filename.replace(".xlsx", "_subjectivity_scored.xlsx")
df.to_excel(output_file, index=False)
files.download(output_file)

print("✅ 완료: Subjectivity_score (0~1)가 계산되어 저장되었습니다.")

# 6. 종속변수 측정 코드

6.1 RoBERTa 기반 감성분석 측정 코드

In [ ]:
# 📚 라이브러리
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
from tqdm import tqdm
from google.colab import files

# ✅ 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 📂 파일 업로드
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# 📄 데이터 로딩
df = pd.read_excel(filename)
if 'Review_Raw' not in df.columns:
    raise ValueError("❌ 'Review' 열이 없습니다.")
df['Review_Raw'] = df['Review_Raw'].fillna("")

# 🤖 모델 로드
MODEL = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL).to(device)

# ✅ 감성 분석 함수 (batch)
def analyze_sentiment_batch(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    probs = softmax(outputs.logits.cpu().numpy(), axis=1)
    return probs  # shape: (batch_size, 3)

# 🔄 배치 감성 분석
batch_size = 32
scores = []

for i in tqdm(range(0, len(df), batch_size)):
    batch_texts = df['Review_Raw'].iloc[i:i+batch_size].tolist()
    try:
        probs = analyze_sentiment_batch(batch_texts)
        for prob in probs:
            pos, neu, neg = prob[2], prob[1], prob[0]
            # compound 계산
            compound = round((pos - neg) * (1 - neu), 4)
            scores.append(compound)
    except Exception as e:
        print(f"❌ 오류 at batch {i}: {e}")
        scores.extend([0.0] * len(batch_texts))

df['sentiment_score'] = scores

# ✅ 분위수 계산 (25%, 75%)
q25 = df['sentiment_score'].quantile(0.25)
q75 = df['sentiment_score'].quantile(0.75)

print(f"🎯 분위수 기준: Q25={q25:.4f}, Q75={q75:.4f}")

# 🔄 분위수 기반 라벨링 함수
def label_sentiment(score):
    if score <= q25:
        return 2  # Negative
    elif score >= q75:
        return 1  # Positive
    else:
        return 3  # Neutral

df['sentiment_label'] = df['sentiment_score'].apply(label_sentiment)

# 💾 저장 및 다운로드
output_file = filename.replace(".xlsx", "_roberta_sentiment_labeled.xlsx")
df.to_excel(output_file, index=False)
files.download(output_file)

print(f"✅ 완료: RoBERTa 감성 분석 + 분위수 라벨링 (1=Positive≥{q75:.2f}, 2=Negative≤{q25:.2f}, 3=Neutral)")
